# Museum Analysis

In [1]:
import os
import pandas as pd
from psycopg2 import connect, Error
from psycopg2.extras import RealDictCursor
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt

In [2]:
DB_HOST = os.getenv('DB_HOST')
DB_PORT = os.getenv('DB_PORT')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

In [3]:
conn = connect(
        dbname=DB_NAME,
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        cursor_factory=RealDictCursor
    )

In [4]:
def run_sql_query(query: str) -> None:
    try:
        connection = connect(
        dbname=DB_NAME,
        host=DB_HOST,
        port=DB_PORT,
        user=DB_USER,
        password=DB_PASSWORD,
        cursor_factory=RealDictCursor
        )
        cursor = connection.cursor()
        connection.autocommit = False

        cursor.execute(query)
        connection.commit()

        rows = cursor.fetchall()
        result = [dict(row) for row in rows]
        print(result)
        print(rows)

    except Error as e:
        print(e)
        connection.rollback()
        
    finally:
        cursor.close()
        connection.close()
        return result

## What exhibition is most frequently visited?

In [5]:
most_visited_exhibition = """
    SELECT exhibit_id, SUM(event_count) AS total_count
    FROM (
    SELECT exhibit_id, COUNT(id) AS event_count
    FROM rating_events
    GROUP BY exhibit_id
    
    UNION ALL
    
    SELECT exhibit_id, COUNT(support_value_id) AS event_count
    FROM support_events
    GROUP BY exhibit_id
    ) AS combined_events
    GROUP BY exhibit_id
    ORDER BY total_count DESC
    LIMIT 1;
    """

query_result = run_sql_query(most_visited_exhibition)
f'The most frequently visited exhibition is exhibit {query_result[0]["exhibit_id"]}'

[{'exhibit_id': 4, 'total_count': Decimal('16032')}]
[RealDictRow([('exhibit_id', 4), ('total_count', Decimal('16032'))])]


'The most frequently visited exhibition is exhibit 4'

## What hour of the day has the most ratings?

In [31]:
most_popular_hour = """
    SELECT EXTRACT(HOUR FROM rated_at) as hour,
    COUNT(rating_value_id) AS ratings
    FROM rating_events
    GROUP BY hour
    ORDER BY ratings DESC;
"""

query = run_sql_query(most_popular_hour)

for row in query:
    print(f"Hour: {row['hour']} Ratings: {row['ratings']}")

[{'hour': Decimal('10'), 'ratings': 7424}, {'hour': Decimal('13'), 'ratings': 7296}, {'hour': Decimal('17'), 'ratings': 7200}, {'hour': Decimal('14'), 'ratings': 7168}, {'hour': Decimal('9'), 'ratings': 7072}, {'hour': Decimal('16'), 'ratings': 6688}, {'hour': Decimal('11'), 'ratings': 6272}, {'hour': Decimal('15'), 'ratings': 5952}, {'hour': Decimal('12'), 'ratings': 5728}, {'hour': Decimal('8'), 'ratings': 3232}, {'hour': Decimal('18'), 'ratings': 2016}]
[RealDictRow([('hour', Decimal('10')), ('ratings', 7424)]), RealDictRow([('hour', Decimal('13')), ('ratings', 7296)]), RealDictRow([('hour', Decimal('17')), ('ratings', 7200)]), RealDictRow([('hour', Decimal('14')), ('ratings', 7168)]), RealDictRow([('hour', Decimal('9')), ('ratings', 7072)]), RealDictRow([('hour', Decimal('16')), ('ratings', 6688)]), RealDictRow([('hour', Decimal('11')), ('ratings', 6272)]), RealDictRow([('hour', Decimal('15')), ('ratings', 5952)]), RealDictRow([('hour', Decimal('12')), ('ratings', 5728)]), RealDict

## What is the average rating for each exhibition?

In [34]:
average_exhibition_ratings = """
    SELECT exhibit_id, ROUND(AVG(rating_value_id), 2) as average_rating
    FROM rating_events 
    GROUP BY exhibit_id
    ORDER BY average_rating DESC
"""

query = run_sql_query(average_exhibition_ratings)
for row in query:
    print(f"Exhibit: {row['exhibit_id']} Average Rating: {row['average_rating']}")

[{'exhibit_id': 4, 'average_rating': Decimal('3.83')}, {'exhibit_id': 2, 'average_rating': Decimal('2.93')}, {'exhibit_id': 1, 'average_rating': Decimal('2.92')}, {'exhibit_id': 3, 'average_rating': Decimal('2.43')}, {'exhibit_id': 5, 'average_rating': Decimal('2.22')}]
[RealDictRow([('exhibit_id', 4), ('average_rating', Decimal('3.83'))]), RealDictRow([('exhibit_id', 2), ('average_rating', Decimal('2.93'))]), RealDictRow([('exhibit_id', 1), ('average_rating', Decimal('2.92'))]), RealDictRow([('exhibit_id', 3), ('average_rating', Decimal('2.43'))]), RealDictRow([('exhibit_id', 5), ('average_rating', Decimal('2.22'))])]
Exhibit: 4 Average Rating: 3.83
Exhibit: 2 Average Rating: 2.93
Exhibit: 1 Average Rating: 2.92
Exhibit: 3 Average Rating: 2.43
Exhibit: 5 Average Rating: 2.22


## What proportion of all 4+ ratings are given to exhibition 4?

In [35]:
exhibit_four_ratings_proportion = """
    SELECT exhibit_id, 
    ROUND((COUNT(*) * 100.0 / SUM(COUNT(*)) OVER ()), 2) AS proportion_percentage
    FROM rating_events
    WHERE rating_value_id >= 4
    GROUP BY exhibit_id
"""
query = run_sql_query(exhibit_four_ratings_proportion)

for row in query:
    print(f"Exhibit {row['exhibit_id']} has {row['proportion_percentage']}% of ratings with value 4 or higher.")

[{'exhibit_id': 1, 'proportion_percentage': Decimal('15.07')}, {'exhibit_id': 3, 'proportion_percentage': Decimal('12.79')}, {'exhibit_id': 5, 'proportion_percentage': Decimal('9.13')}, {'exhibit_id': 4, 'proportion_percentage': Decimal('44.75')}, {'exhibit_id': 2, 'proportion_percentage': Decimal('18.26')}]
[RealDictRow([('exhibit_id', 1), ('proportion_percentage', Decimal('15.07'))]), RealDictRow([('exhibit_id', 3), ('proportion_percentage', Decimal('12.79'))]), RealDictRow([('exhibit_id', 5), ('proportion_percentage', Decimal('9.13'))]), RealDictRow([('exhibit_id', 4), ('proportion_percentage', Decimal('44.75'))]), RealDictRow([('exhibit_id', 2), ('proportion_percentage', Decimal('18.26'))])]
Exhibit 1 has 15.07% of ratings with value 4 or higher.
Exhibit 3 has 12.79% of ratings with value 4 or higher.
Exhibit 5 has 9.13% of ratings with value 4 or higher.
Exhibit 4 has 44.75% of ratings with value 4 or higher.
Exhibit 2 has 18.26% of ratings with value 4 or higher.


## Are positive ratings more frequent before or after 1pm?

In [36]:
ratings_before_or_after_1pm = """
    SELECT COUNT(CASE WHEN EXTRACT(HOUR FROM rated_at) < 13 THEN 1 END) AS before_1pm,
           COUNT(CASE WHEN EXTRACT(HOUR FROM rated_at) >= 13 THEN 1 END) AS after_1pm
           FROM rating_events
           WHERE rating_value_id >= (
           SELECT AVG(rating_value_id) FROM rating_events
           );
"""
query = run_sql_query(ratings_before_or_after_1pm)

for row in query:
    print(f"Ratings before 1pm: {row['before_1pm']}, Ratings after 1pm: {row['after_1pm']}")

[{'before_1pm': 18144, 'after_1pm': 20640}]
[RealDictRow([('before_1pm', 18144), ('after_1pm', 20640)])]
Ratings before 1pm: 18144, Ratings after 1pm: 20640


## How many ratings each hour are above the average rating for exhibition 4?

In [38]:
hourly_ratings_above_exhibit_4 = """
    SELECT EXTRACT(HOUR FROM rated_at) AS hour,
    COUNT(rating_value_id) AS ratings
    FROM rating_events
    WHERE rating_value_id > (
        SELECT AVG(rating_value_id)
        FROM rating_events
        WHERE exhibit_id = 4
    )
    GROUP BY hour
    ORDER BY hour;
"""

query = run_sql_query(hourly_ratings_above_exhibit_4)
print("Ratings each hour above exhibit 4 average")
for row in query:
    print(f"Hour: {row['hour']}, Ratings: {row['ratings']}")

[{'hour': Decimal('8'), 'ratings': 1120}, {'hour': Decimal('9'), 'ratings': 2208}, {'hour': Decimal('10'), 'ratings': 2368}, {'hour': Decimal('11'), 'ratings': 2304}, {'hour': Decimal('12'), 'ratings': 2016}, {'hour': Decimal('13'), 'ratings': 2240}, {'hour': Decimal('14'), 'ratings': 1920}, {'hour': Decimal('15'), 'ratings': 1728}, {'hour': Decimal('16'), 'ratings': 2272}, {'hour': Decimal('17'), 'ratings': 2240}, {'hour': Decimal('18'), 'ratings': 608}]
[RealDictRow([('hour', Decimal('8')), ('ratings', 1120)]), RealDictRow([('hour', Decimal('9')), ('ratings', 2208)]), RealDictRow([('hour', Decimal('10')), ('ratings', 2368)]), RealDictRow([('hour', Decimal('11')), ('ratings', 2304)]), RealDictRow([('hour', Decimal('12')), ('ratings', 2016)]), RealDictRow([('hour', Decimal('13')), ('ratings', 2240)]), RealDictRow([('hour', Decimal('14')), ('ratings', 1920)]), RealDictRow([('hour', Decimal('15')), ('ratings', 1728)]), RealDictRow([('hour', Decimal('16')), ('ratings', 2272)]), RealDictRo

## Do Zoology exhibitions get better ratings than other types?

In [29]:
above_average_departments = """
    SELECT departments.name, AVG(rating_events.rating_value_id)
    FROM departments
    JOIN exhibitions ON departments.id = exhibitions.department_id
    JOIN rating_events ON exhibitions.id = rating_events.exhibit_id
    GROUP BY departments.id
"""

run_sql_query(above_average_departments)

[{'name': 'Ecology', 'avg': Decimal('2.2162162162162162')}, {'name': 'Zoology', 'avg': Decimal('3.1995332555425904')}, {'name': 'Geology', 'avg': Decimal('2.9169435215946844')}, {'name': 'Entomology', 'avg': Decimal('2.9294117647058824')}]
[RealDictRow([('name', 'Ecology'), ('avg', Decimal('2.2162162162162162'))]), RealDictRow([('name', 'Zoology'), ('avg', Decimal('3.1995332555425904'))]), RealDictRow([('name', 'Geology'), ('avg', Decimal('2.9169435215946844'))]), RealDictRow([('name', 'Entomology'), ('avg', Decimal('2.9294117647058824'))])]


[{'name': 'Ecology', 'avg': Decimal('2.2162162162162162')},
 {'name': 'Zoology', 'avg': Decimal('3.1995332555425904')},
 {'name': 'Geology', 'avg': Decimal('2.9169435215946844')},
 {'name': 'Entomology', 'avg': Decimal('2.9294117647058824')}]

## What exhibition has the most emergencies?

In [11]:
exhibit_with_most_emergencies = """
    SELECT exhibit_id, COUNT(support_value_id) 
    FROM support_events
    WHERE support_value_id = 2
    GROUP BY exhibit_id;
"""

run_sql_query(exhibit_with_most_emergencies)

[{'exhibit_id': 3, 'count': 64}]
[RealDictRow([('exhibit_id', 3), ('count', 64)])]


[{'exhibit_id': 3, 'count': 64}]

## Which exhibitions receive fewer assistance requests than the average?

In [12]:
assistance_requests_per_exhibit = """
    SELECT exhibit_id, COUNT(support_value_id)
    FROM support_events
    WHERE support_value_id = 1
    GROUP BY exhibit_id;
"""

run_sql_query(assistance_requests_per_exhibit)

[{'exhibit_id': 1, 'count': 288}, {'exhibit_id': 3, 'count': 384}, {'exhibit_id': 5, 'count': 352}, {'exhibit_id': 4, 'count': 928}, {'exhibit_id': 2, 'count': 320}]
[RealDictRow([('exhibit_id', 1), ('count', 288)]), RealDictRow([('exhibit_id', 3), ('count', 384)]), RealDictRow([('exhibit_id', 5), ('count', 352)]), RealDictRow([('exhibit_id', 4), ('count', 928)]), RealDictRow([('exhibit_id', 2), ('count', 320)])]


[{'exhibit_id': 1, 'count': 288},
 {'exhibit_id': 3, 'count': 384},
 {'exhibit_id': 5, 'count': 352},
 {'exhibit_id': 4, 'count': 928},
 {'exhibit_id': 2, 'count': 320}]

## Are there particular times when assistance requests/emergencies are more likely?

In [39]:
assistance_requests_per_hour = """
    SELECT EXTRACT (HOUR FROM made_at) AS hour,
    COUNT(support_value_id) AS requests
    FROM support_events
    WHERE support_value_id = 1
    GROUP BY hour
    ORDER BY requests DESC;
"""

query = run_sql_query(assistance_requests_per_hour)

for row in query:
    print(f"Hour: {row['hour']} Requests: {row['requests']}")

[{'hour': Decimal('14'), 'requests': 352}, {'hour': Decimal('16'), 'requests': 288}, {'hour': Decimal('10'), 'requests': 256}, {'hour': Decimal('12'), 'requests': 224}, {'hour': Decimal('17'), 'requests': 224}, {'hour': Decimal('15'), 'requests': 192}, {'hour': Decimal('9'), 'requests': 192}, {'hour': Decimal('11'), 'requests': 192}, {'hour': Decimal('18'), 'requests': 160}, {'hour': Decimal('13'), 'requests': 128}, {'hour': Decimal('8'), 'requests': 64}]
[RealDictRow([('hour', Decimal('14')), ('requests', 352)]), RealDictRow([('hour', Decimal('16')), ('requests', 288)]), RealDictRow([('hour', Decimal('10')), ('requests', 256)]), RealDictRow([('hour', Decimal('12')), ('requests', 224)]), RealDictRow([('hour', Decimal('17')), ('requests', 224)]), RealDictRow([('hour', Decimal('15')), ('requests', 192)]), RealDictRow([('hour', Decimal('9')), ('requests', 192)]), RealDictRow([('hour', Decimal('11')), ('requests', 192)]), RealDictRow([('hour', Decimal('18')), ('requests', 160)]), RealDictR

## Are emergencies more likely in exhibitions that are more exciting than others?

In [40]:
above_average_exhibit_emergencies = """
    SELECT exhibit_id, 
    COUNT(CASE WHEN support_value_id = 2 THEN support_value_id END)
    FROM support_events
    WHERE exhibit_id IN (
        SELECT exhibit_id FROM rating_events
        GROUP BY exhibit_id
        HAVING AVG(rating_value_id) > (SELECT AVG(rating_value_id) FROM rating_events)
    )
    GROUP BY exhibit_id;
"""

query = run_sql_query(above_average_exhibit_emergencies)
print("Emergencies in above average exhibits")
for row in query:
    print(f"Exhibit: {row['exhibit_id']} Emergencies: {row['count']}")

[{'exhibit_id': 1, 'count': 0}, {'exhibit_id': 4, 'count': 0}, {'exhibit_id': 2, 'count': 0}]
[RealDictRow([('exhibit_id', 1), ('count', 0)]), RealDictRow([('exhibit_id', 4), ('count', 0)]), RealDictRow([('exhibit_id', 2), ('count', 0)])]
Emergencies in above average exhibits
Exhibit: 1 Emergencies: 0
Exhibit: 4 Emergencies: 0
Exhibit: 2 Emergencies: 0


## Which floors are above average for ratings?

In [41]:
above_average_floors = """
    SELECT floors.name, AVG(rating_events.rating_value_id)
    FROM floors
    JOIN exhibitions ON floors.id = exhibitions.floor_id
    JOIN rating_events ON exhibitions.id = rating_events.exhibit_id
    GROUP BY floors.id
"""

query = run_sql_query(above_average_floors)

for row in query:
    print(f"Floor: {row['name']} Average Rating: {row['avg']}")

[{'name': '3', 'avg': Decimal('2.2162162162162162')}, {'name': '1', 'avg': Decimal('3.4747736093143596')}, {'name': '2', 'avg': Decimal('2.4259740259740260')}, {'name': 'Vault', 'avg': Decimal('2.9294117647058824')}]
[RealDictRow([('name', '3'), ('avg', Decimal('2.2162162162162162'))]), RealDictRow([('name', '1'), ('avg', Decimal('3.4747736093143596'))]), RealDictRow([('name', '2'), ('avg', Decimal('2.4259740259740260'))]), RealDictRow([('name', 'Vault'), ('avg', Decimal('2.9294117647058824'))])]
Floor: 3 Average Rating: 2.2162162162162162
Floor: 1 Average Rating: 3.4747736093143596
Floor: 2 Average Rating: 2.4259740259740260
Floor: Vault Average Rating: 2.9294117647058824
